#### 1. Instalación de librerías

In [39]:
!pip install faiss-cpu transformers sentence-transformers

#### 2. Cargar corpus de documentos
Debido a restricciones por parte de las páginas oficiales de la Universidad Nacional de Colombia, se crea el siguiente corpus de documentos para ilustrar el RAG

In [67]:
documentos = [
  "La presión por aumentar la estatura del caballo criollo colombiano ha llevado al uso excesivo de hormonas y vitaminas, poniendo en riesgo su bienestar animal.",  # :contentReference[oaicite:1]{index=1}
  "En el Atlántico, científicos detectaron 49 variantes genéticas en personas de 45 a 70 años que podrían anticipar riesgos de Alzheimer, Parkinson u otras demencias.",  # :contentReference[oaicite:2]{index=2}
  "Un estudio concluye que la represión policial hacia consumidores de bazuco es ineficaz y vulnera derechos humanos, recomendando un enfoque de salud pública.",  # :contentReference[oaicite:3]{index=3}
  "Investigadores en Antioquia desarrollaron un sistema con inteligencia artificial y satélites para predecir zonas críticas de deforestación hasta con dos años de antelación.",  # :contentReference[oaicite:4]{index=4}
  "Se implementaron 29 aulas STEAM en regiones apartadas, incluyendo San Basilio de Palenque, para fortalecer la educación en ciencia y tecnología.",  # :contentReference[oaicite:5]{index=5}
  "La alimentación de gallinas criollas con semillas de lino aumentó significativamente los niveles de omega‑3 en los huevos, mejorando su valor nutricional.",  # :contentReference[oaicite:6]{index=6}
]


#### 3. Vectorizar y construir el índice FAISS

In [53]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Cargar modelo de embeddings local
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Vectorizar el corpus
vectores = modelo_embeddings.encode(documentos, show_progress_bar=True)

# Crear el índice FAISS
index = faiss.IndexFlatL2(vectores.shape[1])
index.add(np.array(vectores))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 4. Función para recuperar múltiples contextos

In [54]:
def buscar_contexto(pregunta, top_k=3):
    pregunta_vec = modelo_embeddings.encode([pregunta])
    _, indices = index.search(np.array(pregunta_vec), top_k)
    fragmentos = [documentos[i] for i in indices[0]]
    return " ".join(fragmentos)


#### 5. Cargar modelo generativo FLAN-T5 optimizado para QA

In [55]:
from transformers import pipeline

# Modelo multilingüe, sigue instrucciones y responde preguntas
generador = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


#### 6. Función de respuesta mejorada con prompt claro

In [57]:
def responder(pregunta):
    contexto = buscar_contexto(pregunta, top_k=3)
    prompt = f"Contesta la siguiente pregunta usando solo la información del contexto.\n\nContexto: {contexto}\nPregunta: {pregunta}"
    resultado = generador(prompt, max_new_tokens=100)[0]['generated_text']
    return resultado.strip()

#### 7. Probar el sistema

In [68]:
print(responder("¿Cuántas aulas STEM hay?"))

29


In [76]:
print(responder("¿Puede alimentarse a la gallina con semillas de lino?"))

criollas con semillas de lino aumentó significativamente los niveles de omega3 en los huevos, mejorando su valor nutricional


In [77]:
print(responder("¿Cuentame noticias de la Universidad Nacional de Colombia?"))

la presión para aumentar la estatura del caballo criollo colombiano ha llevado al uso excesivo de hormonas y vitaminas, poniendo en riesgo su bienestar animal
